#### Set styling for plotting

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
sns.set_palette('colorblind')
from matplotlib.pyplot import tight_layout
# ##SETTING PARAMS FOR MATPLOTLIB FIGURES
plt.rcParams.update({"figure.figsize": (6, 6),
                 "axes.facecolor": "white",
                 "axes.edgecolor": "black"})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=sns.color_palette('colorblind'))
##set font size
font = {'family': 'sans-serif',
       'weight': 'normal',
       'size': 14}
plt.rc('font', **font)
# ##PANDAS PLOTTING
pd.plotting.register_matplotlib_converters()

### Step 1: save environment file

In [2]:
!conda env export > combined_met_environment.yml

### Step 2: import modules

In [3]:
import glob
import os
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
from tqdm import tqdm
import codecs
import csv
import pytz
import re
import pandas as pd

### Step 3: import files

In [4]:
data_directory_xl = "/home/paulharford/college/project/project_data/met_eireann/National warnings from pdfs_2023.xlsx"
full_path_xl = os.path.abspath(data_directory_xl)

In [5]:
# Read the 2023 excel file
df_xl = pd.read_excel(full_path_xl)

In [6]:
df_xl.head()

,Unnamed: 0,Unnamed: 1,unprotect cafo,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Ulster,Unnamed: 40,Unnamed: 41,Unnamed: 42
0,Total,NaN,NaN,Yellow,Orange,Red,Named,NaN,Ireland,NaN,...,MH,OY,WH,WX,WW,NaN,All,DL,CN,MN
1,2023-01-01 00:00:00,Fog/Ice,Level: Yellow,243,50,6,0,NaN,0,NaN,...,0,0,0,0,0,NaN,0,0,0,0
2,NaN,NaN,Type: Fog / Ice,0,0,0,0,NaN,0,NaN,...,0,0,0,0,0,NaN,0,0,0,0
3,NaN,NaN,Message: Icy stretches along with patches of f...,0,0,0,0,NaN,0,NaN,...,0,0,0,0,0,NaN,0,0,0,0
4,NaN,NaN,Affected Regions: ireland,0,0,0,0,NaN,1,NaN,...,0,0,0,0,0,NaN,0,0,0,0


In [7]:
# If your DataFrame is named df, you can filter like this:
yellow_events = df_xl.loc[df_xl['Unnamed: 3'] == 1, 'unprotect cafo']
orange_events = df_xl.loc[df_xl['Unnamed: 4'] == 1, 'unprotect cafo']

# To display the list of texts for yellow events:
#print(yellow_events.to_string(index=False))
yellow_events.count()

244

In [8]:
# Filter out events that contain the string "Level: Yellow"
yellow_events_filtered = yellow_events[~yellow_events.str.contains("Level: Yellow", na=False)]

# Display the filtered events (the anomalies)
print("\nAnomalous yellow events (without 'Level: Yellow'):")
print(yellow_events_filtered.to_string(index=False))


Anomalous yellow events (without 'Level: Yellow'):
Status Yellow - Wind warning for Kerry, Clare, ...
Status Yellow - Thunderstorm warning for Clare,...
Status Yellow - Rain warning for Donegal and Le...
          Status Yellow - Wind warning for Donegal
           Status Yellow - Ice warning for Ireland
Status Yellow Thunderstorm Warning issued Poten...
                                            Yellow
   Status Yellow - Wind & Rain warning for Ireland
           Status Yellow - Fog warning for Ireland
                                            yellow


In [9]:
# Filter out events that contain the string "Level: Yellow"
orange_events_filtered = orange_events[~orange_events.str.contains("Level: Orange", na=False)]

# Display the filtered events (the anomalies)
print("\nAnomalous yellow events (without 'Level: Orange'):")
print(orange_events_filtered.to_string(index=False))


Anomalous yellow events (without 'Level: Orange'):
       Status Orange - Wind warning for Donegal
Status Orange - Rain warning for Cork and Kerry


### Step 4: process Excel data 

In [10]:
def standardize_datetime(date_str):
    try:
        if pd.isna(date_str):
            return None
            
        date_str = str(date_str).strip()
        
        # Handle special cases first
        if '/' in date_str and 'updated' in date_str.lower():
            date_str = date_str.split('/')[0].strip()
        
        # Extract the basic date and time using regex
        basic_date_match = re.match(r'([A-Za-z]+\s+\d{1,2}/\d{1,2}/\d{4}\s+\d{1,2}[:\.]\d{2})', date_str)
        
        if basic_date_match:
            basic_date_str = basic_date_match.group(1)
            
            # Try to parse with the most common format first
            try:
                dt = datetime.strptime(basic_date_str, "%A %d/%m/%Y %H:%M")
                return pytz.utc.localize(dt).isoformat()
            except ValueError:
                # If that fails, try with period instead of colon
                try:
                    dt = datetime.strptime(basic_date_str, "%A %d/%m/%Y %H.%M")
                    return pytz.utc.localize(dt).isoformat()
                except ValueError:
                    pass
        
        # If the regex approach didn't work, try the original approach with your format list
        if len(date_str.split()) < 3:  # Skip incomplete dates
            return None
           
        formats = [
            "%A %d/%m/%Y %H:%M",
            "%H:%M %A %d/%m/%Y",
            "%A %d/%m/%Y %H.%M", 
            "%d/%m/%Y %H:%M",
            "%d/%m/%Y %H.%M",
            "%A %d/%m/%Y"  # Format without time
        ]
       
        for fmt in formats:
            try:
                dt = datetime.strptime(date_str, fmt)
                return pytz.utc.localize(dt).isoformat()
            except ValueError:
                continue
        

        pattern1 = r'([A-Za-z]+)\s+(\d{1,2}/\d{1,2}/\d{4})\s+(\d{1,2}[:\.]\d{2})'
        match1 = re.match(pattern1, date_str)
        if match1:
            day_name, date_part, time_part = match1.groups()
            time_part = time_part.replace('.', ':')  # Standardize to use colons
            try:
                dt = datetime.strptime(f"{day_name} {date_part} {time_part}", "%A %d/%m/%Y %H:%M")
                return pytz.utc.localize(dt).isoformat()
            except ValueError:
                pass
        

        try:
            dt = pd.to_datetime(date_str, errors='coerce')
            if pd.notna(dt):
                return dt.tz_localize('UTC').isoformat()
        except:
            pass
        
        return None
    except Exception as e:
        print(f"Error converting date {date_str}: {str(e)}")
        return None

def transform_warnings_data(df):
    processed_data = []
    current_event = None
    event_texts = []  
    
    # Define the counties (used for affected regions)
    counties = {
        'Carlow': 0, 'Cavan': 0, 'Clare': 0, 'Cork': 0, 'Donegal': 0,
        'Dublin': 0, 'Galway': 0, 'Kerry': 0, 'Kildare': 0, 'Kilkenny': 0,
        'Laois': 0, 'Leitrim': 0, 'Limerick': 0, 'Longford': 0, 'Louth': 0,
        'Mayo': 0, 'Meath': 0, 'Monaghan': 0, 'Offaly': 0, 'Roscommon': 0,
        'Sligo': 0, 'Tipperary': 0, 'Waterford': 0, 'Westmeath': 0,
        'Wexford': 0, 'Wicklow': 0
    }
    
    county_map = {
        'CK': 'Cork', 'CE': 'Clare', 'KY': 'Kerry', 'LK': 'Limerick', 
        'TY': 'Tipperary', 'WD': 'Waterford', 'GY': 'Galway', 'LM': 'Leitrim',
        'MO': 'Mayo', 'RN': 'Roscommon', 'SO': 'Sligo', 'CW': 'Carlow',
        'DN': 'Dublin', 'KE': 'Kildare', 'KK': 'Kilkenny', 'LS': 'Laois',
        'LD': 'Longford', 'LH': 'Louth', 'MH': 'Meath', 'OY': 'Offaly',
        'WH': 'Westmeath', 'WX': 'Wexford', 'WW': 'Wicklow', 'DL': 'Donegal',
        'CN': 'Cavan', 'MN': 'Monaghan'
    }
    
    def safe_float_convert(value, default=0):
        """Safely convert a value to float, returning default if conversion fails."""
        if pd.isna(value):
            return default
        try:
            return float(value)
        except (ValueError, TypeError):
            return default
    
    def detect_warning_color(text):
        """Detect warning color from text with multiple pattern recognition."""
        text_lower = text.lower()
        
        # Check for various patterns indicating red warnings
        if any(pattern in text_lower for pattern in [
            'level: red', 'status red', 'red warning', 'red level', 'red status',
            'red alert', 'weather warning: red'
        ]):
            return 'Red'
            
        # Check for various patterns indicating orange warnings
        elif any(pattern in text_lower for pattern in [
            'level: orange', 'status orange', 'orange warning', 'orange level', 
            'orange status', 'orange alert', 'weather warning: orange'
        ]):
            return 'Orange'
            
        # Check for various patterns indicating yellow warnings
        elif any(pattern in text_lower for pattern in [
            'level: yellow', 'status yellow', 'yellow warning', 'yellow level',
            'yellow status', 'yellow alert', 'weather warning: yellow'
        ]):
            return 'Yellow'
        
        return None
    
    # Iterate over all rows in the DataFrame.
    for _, row in df.iterrows():
        # A new event starts when 'Unnamed: 1' is not null.
        if pd.notna(row['Unnamed: 1']):
            # Finalize the previous event (if any)
            if current_event is not None:
                combined_text = " ".join(event_texts).lower()
                
                # Save the full combined text for debugging, if needed.
                current_event['Combined Text'] = " ".join(event_texts)
                
                # Skip events that contain the word "advisory"
                if "advisory" not in combined_text:
                    # First check if we already assigned a color during parsing
                    if 'Warning Colour' not in current_event:
                        # Try to detect color from the combined text
                        color = detect_warning_color(combined_text)
                        if color:
                            current_event['Warning Colour'] = color
                    
                    # If still no color, use flag values in combination with general text mentions
                    if 'Warning Colour' not in current_event:
                        # Check for any mention of color words
                        has_red = 'red' in combined_text
                        has_orange = 'orange' in combined_text
                        has_yellow = 'yellow' in combined_text
                        
                        # Use flag values to determine which one to use
                        if current_event.get('flag_red', 0) > 0 and has_red:
                            current_event['Warning Colour'] = 'Red'
                        elif current_event.get('flag_orange', 0) > 0 and has_orange:
                            current_event['Warning Colour'] = 'Orange'
                        elif current_event.get('flag_yellow', 0) > 0 and has_yellow:
                            current_event['Warning Colour'] = 'Yellow'
                        # If flags alone are set but no color mentioned, use highest priority flag
                        elif current_event.get('flag_red', 0) > 0:
                            current_event['Warning Colour'] = 'Red'
                        elif current_event.get('flag_orange', 0) > 0:
                            current_event['Warning Colour'] = 'Orange'
                        elif current_event.get('flag_yellow', 0) > 0:
                            current_event['Warning Colour'] = 'Yellow'
                    
                    processed_data.append(current_event.copy())
            
            # Start a new event.
            current_event = {}
            event_texts = []  # Reset the accumulator
            
            # Use the value from 'Unnamed: 1' as the initial Warning Element.
            current_event['Warning Element'] = row['Unnamed: 1']
            current_event.update(counties.copy())
            
            # Retrieve the columns for yellow, orange, and red with error handling
            current_event['flag_yellow'] = safe_float_convert(row['Unnamed: 3'])
            current_event['flag_orange'] = safe_float_convert(row['Unnamed: 5'])
            current_event['flag_red'] = safe_float_convert(row['Unnamed: 5'])
            
            # If an "Ireland" column exists and equals 1, mark all counties.
            if 'Ireland' in row.index:
                try:
                    if float(row['Ireland']) == 1:
                        for county in counties.keys():
                            current_event[county] = 1
                except Exception:
                    pass
            
            # Process the current row's text from "unprotect cafo"
            text_line = str(row['unprotect cafo']) if pd.notna(row['unprotect cafo']) else ""
            event_texts.append(text_line)
            
            # Check for warning color in the text
            color = detect_warning_color(text_line)
            if color:
                current_event['Warning Colour'] = color
            
            # Continue with field-specific parsing
            field = text_line.lower()
            if 'type:' in field:
                current_event['Warning Element'] = text_line.replace('Type:', '').strip()
            elif 'message:' in field:
                current_event['Warning Text'] = text_line.replace('Message:', '').strip()
            elif 'issue time:' in field:
                current_event['Issue Time'] = standardize_datetime(text_line.replace('Issue Time:', '').strip())
            elif 'expected onset:' in field:
                current_event['Valid From'] = standardize_datetime(text_line.replace('Expected Onset:', '').strip())
            elif 'expires:' in field:
                current_event['Valid To'] = standardize_datetime(text_line.replace('Expires:', '').strip())
            elif 'affected regions:' in field:
                regions = text_line.replace('Affected Regions:', '').strip()
                if 'ireland' in regions.lower():
                    for county in counties.keys():
                        current_event[county] = 1
                else:
                    for abbr, full_name in county_map.items():
                        if abbr in row.index and row[abbr] == 1:
                            current_event[full_name] = 1
        else:
            # Continuation row for the current event.
            if current_event is not None:
                text_line = str(row['unprotect cafo']) if pd.notna(row['unprotect cafo']) else ""
                event_texts.append(text_line)
                
                # Check for warning color in the text
                color = detect_warning_color(text_line)
                if color and 'Warning Colour' not in current_event:
                    current_event['Warning Colour'] = color
                
                # Continue with field-specific parsing
                field = text_line.lower()
                if 'type:' in field:
                    current_event['Warning Element'] = text_line.replace('Type:', '').strip()
                elif 'message:' in field:
                    current_event['Warning Text'] = text_line.replace('Message:', '').strip()
                elif 'issue time:' in field:
                    current_event['Issue Time'] = standardize_datetime(text_line.replace('Issue Time:', '').strip())
                elif 'expected onset:' in field:
                    current_event['Valid From'] = standardize_datetime(text_line.replace('Expected Onset:', '').strip())
                elif 'expires:' in field:
                    current_event['Valid To'] = standardize_datetime(text_line.replace('Expires:', '').strip())
                elif 'affected regions:' in field:
                    regions = text_line.replace('Affected Regions:', '').strip()
                    if 'ireland' in regions.lower():
                        for county in counties.keys():
                            current_event[county] = 1
                    else:
                        for abbr, full_name in county_map.items():
                            if abbr in row.index and row[abbr] == 1:
                                current_event[full_name] = 1
                
                # Also check for the Ireland flag in continuation rows.
                if 'Ireland' in row.index:
                    try:
                        if float(row['Ireland']) == 1:
                            for county in counties.keys():
                                current_event[county] = 1
                    except Exception:
                        pass
                        
    # Finalize the last event (if any)
    if current_event is not None:
        combined_text = " ".join(event_texts).lower()
        current_event['Combined Text'] = " ".join(event_texts)
        
        # Skip events that contain the word "advisory"
        if "advisory" not in combined_text:
            # First check if we already assigned a color during parsing
            if 'Warning Colour' not in current_event:
                # Try to detect color from the combined text
                color = detect_warning_color(combined_text)
                if color:
                    current_event['Warning Colour'] = color
            

            if 'Warning Colour' not in current_event:
                # Check for any mention of color words
                has_red = 'red' in combined_text
                has_orange = 'orange' in combined_text
                has_yellow = 'yellow' in combined_text
                
                # Use flag values to determine which one to use
                if current_event.get('flag_red', 0) > 0 and has_red:
                    current_event['Warning Colour'] = 'Red'
                elif current_event.get('flag_orange', 0) > 0 and has_orange:
                    current_event['Warning Colour'] = 'Orange'
                elif current_event.get('flag_yellow', 0) > 0 and has_yellow:
                    current_event['Warning Colour'] = 'Yellow'
                # If flags alone are set but no color mentioned, use highest priority flag
                elif current_event.get('flag_red', 0) > 0:
                    current_event['Warning Colour'] = 'Red'
                elif current_event.get('flag_orange', 0) > 0:
                    current_event['Warning Colour'] = 'Orange'
                elif current_event.get('flag_yellow', 0) > 0:
                    current_event['Warning Colour'] = 'Yellow'
            
            processed_data.append(current_event.copy())
    
    return pd.DataFrame(processed_data)

In [11]:
df_xlsx = transform_warnings_data(df_xl)

### Step 5 : Check after processing

In [12]:
df_xlsx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Warning Element  299 non-null    object 
 1   Carlow           299 non-null    int64  
 2   Cavan            299 non-null    int64  
 3   Clare            299 non-null    int64  
 4   Cork             299 non-null    int64  
 5   Donegal          299 non-null    int64  
 6   Dublin           299 non-null    int64  
 7   Galway           299 non-null    int64  
 8   Kerry            299 non-null    int64  
 9   Kildare          299 non-null    int64  
 10  Kilkenny         299 non-null    int64  
 11  Laois            299 non-null    int64  
 12  Leitrim          299 non-null    int64  
 13  Limerick         299 non-null    int64  
 14  Longford         299 non-null    int64  
 15  Louth            299 non-null    int64  
 16  Mayo             299 non-null    int64  
 17  Meath           

no provinces listed after processing

In [13]:
warning_counts = df_xlsx['Warning Colour'].value_counts()
print("Warning Counts:")
print(warning_counts)

Warning Counts:
Warning Colour
Yellow    242
Orange     50
Red         6
Name: count, dtype: int64


### Step 6: Save data to file

In [14]:
df_xlsx.to_csv('/home/paulharford/college/project/project_data/met_eireann/xl_warnings_2023.csv', index=False)